# Parameter attacks on Transformers? 

Following the tutorial at https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [1]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

## Model definition

In [3]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

### Positional Embedding

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.0, max_len: int = 5000):
        super().__init__()
        # self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return x # self.dropout(x)

### Data stuff (boring)

In [5]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)



def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 1
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

### Here define the sequence length we want to consider (don't make too long ;) 

In [6]:
bptt = 30

In [7]:
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

src_mask = generate_square_subsequent_mask(bptt).to(device)

### Instantiate transformer

In [8]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.0 # Kinda need this to be zero :/
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

### Get some sample sentence just to see what we're dealing with

In [9]:
data, targets = get_batch(train_data, 100)
data.shape
print('\n'.join([' '.join(vocab.lookup_tokens(data[:, i].tolist())) for i in range(data.shape[1])]))

penal military unit serving the nation of gallia during the second europan war who perform secret black operations and are pitted against the imperial unit <unk> raven . the game


## Attack setup starts here

### First, let's define our linear measurement - some Gaussian vector here

In [10]:
import math
weights = torch.randn(200)
std, mu = torch.std_mean(weights)
measurement = (weights - mu) / std / math.sqrt(200) # Here's our linear measurement
getattr(model.transformer_encoder.layers, '0').self_attn.out_proj.weight.data = torch.zeros_like(getattr(model.transformer_encoder.layers, '0').self_attn.out_proj.weight.data)
# Above we set the attention head to be zero. Probably should use these somehow...

### Setting the second linear layer to just all zeros except for first row helps? Need to figure out why ... 

In [11]:
getattr(model.transformer_encoder.layers, '0').linear2.weight.data = torch.zeros_like(getattr(model.transformer_encoder.layers, '0').linear2.weight.data)
getattr(model.transformer_encoder.layers, '0').linear2.weight.data[0] = torch.ones_like(getattr(model.transformer_encoder.layers, '0').linear2.weight.data[0])
getattr(model.transformer_encoder.layers, '0').linear2.bias.data = torch.zeros_like(getattr(model.transformer_encoder.layers, '0').linear2.bias.data)


### Now let's get the feature statistics for our random measurement vector

In [12]:
def feature_distribution(model):
        """Compute the mean and std of the feature layer of the given network."""
        features = dict()
        setup = dict(device=device)
        def named_hook(name):
            def hook_fn(module, input, output):
                features[name] = input[0]
            return hook_fn
        
        for name, module in list(model.named_modules()):
            if name == 'transformer_encoder.layers.0.linear1':
                print(f'In Linear: {name}')
                hook = module.register_forward_hook(named_hook(name))
                feature_layer_name = name
                break
        feats = []
        feats_before = []
        model.train()
        model.to(**setup)
        print(f'Computing feature distribution before the {feature_layer_name} layer from external data.')
        for batch, i in enumerate(list(range(0, train_data.size(0) - 1, bptt))[:-1]):
            inputs, targets = get_batch(train_data, i)
            inputs = inputs.to(**setup)
            model(inputs, src_mask)
        model.eval()
        for batch, i in enumerate(list(range(0, train_data.size(0) - 1, bptt))[:-1]):
            inputs, targets = get_batch(train_data, i)
            inputs = inputs.to(**setup)
            model(inputs, src_mask)
            feats.append(features[feature_layer_name].detach().view(inputs.shape[0]*inputs.shape[1], -1).clone().cpu())
        std, mu = torch.std_mean(torch.mm(torch.cat(feats), measurement.unsqueeze(1)).squeeze())
        print(f'Feature mean is {mu.item()}, feature std is {std.item()}.')
        model.eval()
        model.cpu()
        hook.remove()

        return std, mu

### Now we'll construct the imprint weights. Only for the first linear layer here...

In [13]:
from statistics import NormalDist

def _get_bins(mean, std, num_bins):
    bins = []
    mass_per_bin = 1 / (num_bins)
    bins.append(-10)  # -Inf is not great here, but NormalDist(mu=0, sigma=1).cdf(10) approx 1
    for i in range(1, num_bins):
        bins.append(NormalDist().inv_cdf(i * mass_per_bin)*std + mean)
    return bins

def _make_biases(bias_layer, bins):
    new_biases = torch.zeros_like(bias_layer.data)
    for i in range(new_biases.shape[0]):
        new_biases[i] = -bins[i]
    return new_biases

feature_std, feature_mean = feature_distribution(model)
bins = _get_bins(feature_mean, feature_std, model.d_model)
getattr(model.transformer_encoder.layers, '0').linear1.weight.data = measurement.repeat(model.d_model, 1)
getattr(model.transformer_encoder.layers, '0').linear1.bias.data = _make_biases(getattr(model.transformer_encoder.layers, '0').linear1.bias, bins)


In Linear: transformer_encoder.layers.0.linear1
Computing feature distribution before the transformer_encoder.layers.0.linear1 layer from external data.
Feature mean is -0.16924509406089783, feature std is 0.9067009687423706.


### Get gradients

In [14]:
criterion = nn.CrossEntropyLoss()
inputs, targets = get_batch(train_data, 100)
#inputs, targets = get_batch(train_data, 0)
inputs = inputs.to(device=device)
model.to(device=device)
model.zero_grad()
output = model(inputs, src_mask)
loss = criterion(output.view(-1, ntokens), targets)
loss.backward()

## Actual attack starts here

### Now get the bag of words, as well as the reconstructed positionally encoded stuff

In [15]:
leaked_tokens = ((model.encoder.weight.grad != 0).sum(dim=1) > 0).nonzero().squeeze() # Bag of words tokens
weight_grad = getattr(model.transformer_encoder.layers, '0').linear1.weight.grad.detach().clone().cpu()
bias_grad = getattr(model.transformer_encoder.layers, '0').linear1.bias.grad.detach().clone().cpu()

for i in reversed(list(range(1, weight_grad.shape[0]))):
    weight_grad[i] -= weight_grad[i - 1]
    bias_grad[i] -= bias_grad[i - 1]
valid_classes = bias_grad != 0

recs = weight_grad[valid_classes, :] / bias_grad[valid_classes, None] # Here are our reconstructed positionally encoded features

### Now the messy stuff ... 

In [19]:
# First, let's get the features of our bag of words sans positional encoding
no_pos = getattr(model.transformer_encoder.layers, '0').norm1((model.encoder(leaked_tokens) * math.sqrt(model.d_model))).cpu()
with_pos = recs # Here are those same features, but with positional encodings (stuff we reconstructed)

import numpy as np
indcs = []
corrs = torch.zeros((len(no_pos), len(with_pos))) 

# We need to find out what word led to what positionally encoded representation. 
# Let's try the naive greedy search for correlations between no_pos and with_pos as defined above
for i, no_p in enumerate(no_pos):
    max_corr = 0
    for j, with_p in enumerate(with_pos):
        val = np.corrcoef(np.array([no_p.detach().numpy(), with_p.detach().numpy()]))[0,1]
        corrs[i,j] = val

# Greedy search to find which positionally-encoded vector associates with un-positionally-encoded vector
order = []
for i in range(len(no_pos)):
    row, col = torch.argmax(corrs) // corrs.shape[1], torch.argmax(corrs) % corrs.shape[1]
    order.append((row.item(), col.item()))
    corrs[row,:] = -1
    corrs[:, col] = -1

order = sorted(order, key=lambda x: x[1])

# Now let's re-sort the tokens by this order
sorted_tokens1 = [leaked_tokens[order[i][0]] for i in range(len(order))]

# Now that we've 'lined-up' the pos-encoded features with non-pos-encoded features, let's subtract the two
# to get some 'faux' positions (layer norm means they aren't exact).
estimated_pos = torch.stack([with_pos[order[i][1]] - no_pos[order[i][0]] for i in range(len(order))])

# Now let's get just the additive part of the positional encoding
just_pos = torch.stack([getattr(model.transformer_encoder.layers, '0').norm1(model.pos_encoder(torch.zeros_like(model.encoder(inputs)) * math.sqrt(model.d_model)))]).cpu().squeeze()

# We'll do another greedy search, but now it's on the positions of the tokens
# First calculate the correlations between (faux) estimated_pos and the (real) just_pos terms
order_coeffs = torch.zeros((len(estimated_pos), len(just_pos)))
for i in range(len(estimated_pos)):
    for j in range(len(just_pos)):
        order_coeffs[i,j] = np.corrcoef(estimated_pos[i].detach().numpy(), just_pos[j].detach().numpy())[0,1]

print(order_coeffs)

# Greedily gobble up the most correlated positions :) 
pos_order = []
for i in range(len(estimated_pos)):
    row, col = torch.argmax(order_coeffs) // order_coeffs.shape[1], torch.argmax(order_coeffs) % order_coeffs.shape[1]
    pos_order.append((row.item(), col.item()))
    order_coeffs[row,:] = -1
    order_coeffs[:, col] = -1

# Finally, reorder one more time ...
pos_order = sorted(pos_order, key=lambda x: x[1])
final_sorted_tokens = [sorted_tokens1[pos_order[i][0]] for i in range(len(pos_order))]


tensor([[0.9086, 0.9654, 0.9091, 0.7721, 0.6410, 0.5520, 0.5071, 0.4940, 0.4929,
         0.4865, 0.4701, 0.4492, 0.4303, 0.4160, 0.4043, 0.3918, 0.3779, 0.3652,
         0.3564, 0.3521, 0.3503, 0.3484, 0.3452, 0.3420, 0.3403, 0.3399, 0.3389,
         0.3355, 0.3299, 0.3245],
        [0.3604, 0.3528, 0.3534, 0.3540, 0.3581, 0.3718, 0.4013, 0.4464, 0.4928,
         0.5210, 0.5275, 0.5319, 0.5643, 0.6433, 0.7567, 0.8608, 0.9088, 0.8820,
         0.7980, 0.6971, 0.6189, 0.5804, 0.5704, 0.5652, 0.5491, 0.5235, 0.5001,
         0.4880, 0.4853, 0.4829],
        [0.3389, 0.3309, 0.3631, 0.4120, 0.4501, 0.4657, 0.4669, 0.4747, 0.5034,
         0.5456, 0.5805, 0.5966, 0.6059, 0.6377, 0.7139, 0.8236, 0.9224, 0.9618,
         0.9205, 0.8177, 0.7020, 0.6214, 0.5918, 0.5934, 0.5936, 0.5742, 0.5415,
         0.5149, 0.5065, 0.5102],
        [0.3116, 0.3121, 0.3217, 0.3356, 0.3557, 0.3800, 0.4042, 0.4247, 0.4393,
         0.4505, 0.4638, 0.4809, 0.4964, 0.5014, 0.4920, 0.4764, 0.4720, 0.4916,
       

### Let's see how we did 

In [17]:
print(' '.join(vocab.lookup_tokens([x[0] for x in inputs]))) # The true sentence
print(' '.join(vocab.lookup_tokens(final_sorted_tokens))) # What we reconstruct

['penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are', 'pitted', 'against', 'the', 'imperial', 'unit', '<unk>', 'raven', '.', 'the', 'game']
['penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'second', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are', 'pitted', 'against', 'imperial', 'europan', '<unk>', 'raven', '.', 'game']
